In [3]:
import numpy as np
from pathlib import Path
import egobox as egx
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# from rich import print
import pickle
from _cad import MyGPX

In [4]:
cfd_dir=Path("./result/CFD/")
cfdData=[]
for i in range(1500):
    filename=cfd_dir/f"{i:03d}.csv"
    tmp=np.loadtxt(fname=filename,delimiter=',',ndmin=2)[:,[0,1,3]]
    if np.any(tmp[:,[0,1]]<0.):
        print(i)
    tmp=tmp.ravel()
    if len(tmp)!=9:
        print(f"{i:03d} is error!")
    else:
        cfdData.append(tmp)
cfdData=np.vstack(cfdData)
cadData=np.loadtxt("./sample.csv",delimiter=',',ndmin=2)
print(cfdData.shape,cadData.shape)

# flag=600
# trainX=cadData[:flag,:]
# testX=cadData[flag:,:]
# trainY=cfdData[:flag,:]
# testY=cfdData[flag:,:]
cfdID=3
X_train, X_test, Y_train, Y_test = train_test_split(cadData, cfdData[:,[cfdID]], test_size = 75,shuffle=False, random_state = 1)
print(X_train.shape,Y_train.shape)

scalerX=MinMaxScaler()
scalerY=MinMaxScaler()

scalerX=scalerX.fit(X_train)
scalerY=scalerY.fit(Y_train)

X_train=scalerX.transform(X_train)
X_test=scalerX.transform(X_test)

Y_train=scalerY.transform(Y_train)
Y_test=scalerY.transform(Y_test)


(1500, 9) (1500, 25)
(1425, 25) (1425, 1)


In [5]:
from smt.surrogate_models import KPLS

sm=KPLS(n_comp=1,print_prediction=False)
sm.set_training_values(xt=X_train,yt=Y_train)
sm.train()

pred_Y=sm.predict_values(X_train)
print("train_error_r2",r2_score(Y_train.ravel(),pred_Y.ravel()))
print("train_error_mean",mean_squared_error(Y_train,pred_Y))

pred_Y=sm.predict_values(X_test)
print("test_error_r2",r2_score(Y_test.ravel(),pred_Y.ravel()))
print("test_error_mean",mean_squared_error(Y_test,pred_Y))


___________________________________________________________________________
   
                                   KPLS
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 1425
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec): 137.2410319
train_error_r2 1.0
train_error_mean 4.434473023827544e-30
test_error_r2 0.35948342697657354
test_error_mean 0.004437948711531421


In [6]:
IDs=[0,799,1460,1462,1464,1466,1470,1475,1480]
cadData_nor=scalerX.transform(cadData)
pred_Y=sm.predict_values(x=cadData_nor[IDs,:])
print(scalerY.inverse_transform(pred_Y).ravel())
print(cfdData[[IDs],[cfdID]].ravel())

[2.384452   2.37981    2.39928462 2.37619637 2.41063803 2.34826263
 2.26046747 2.36751269 2.26254013]
[2.384452 2.37981  2.398755 2.388928 2.407144 2.330423 2.1939   2.336977
 2.389824]


In [26]:
gpx=egx.Gpx.builder(n_clusters=1,regr_spec=7,corr_spec=15,kpls_dim=15,n_start=30,seed=1).fit(X_train,Y_train)

pred_Y=gpx.predict(X_train)
print("train_error_r2",r2_score(Y_train.ravel(),pred_Y.ravel()))
print("train_error_mean",mean_squared_error(Y_train,pred_Y))

pred_Y=gpx.predict(X_test)
print("test_error_r2",r2_score(Y_test.ravel(),pred_Y.ravel()))
print("test_error_mean",mean_squared_error(Y_test,pred_Y))


train_error_r2 1.0

train_error_mean 2.0237112151504553e-25

test_error_r2 0.07088371864094467

test_error_mean 1.0705979597977007

In [24]:
vX=scalerX.transform(cadData)
# vX=cadData
pred_Y=gpx.predict(vX[[0,200,1493,1499],:])
# print(pred_Y)
print(scalerY.inverse_transform(pred_Y))

print(cfdData[[[0,200,1493,1499]],[cfdID]])

[[0.069824  ]
 [0.062209  ]
 [0.07476958]
 [0.10917041]]

[[0.069824 0.062209 0.064394 0.067588]]

In [93]:
mygpx=MyGPX(tag=cfdID,scalerX=scalerX,scalerY=scalerY,gpx=gpx)
mygpx.save()

In [94]:
mygpx=MyGPX(tag=cfdID)
mygpx.load()
pred_Y=mygpx.predict(cadData[[0,1,199,200,399],:])
print(pred_Y)

[[0.113469]
 [0.079558]
 [0.326469]
 [0.392096]
 [0.092398]]